This notebook tries to resolve the stackoverflow http://stackoverflow.com/questions/42286044/get-available-apartments-query question with an adhoc example

## Init Elasticsearch indexes and data

Lets create the index structure and load data

In [2]:
import json
from elasticsearch import Elasticsearch

#cluster node to query
es = Elasticsearch(['localhost:9200',])
index_name    = 'apartments'

#load index_definition
file_path = 'mapping.json'
index_definition = json.load( open( file_path, "rb" ) )

#clean previous tests
es.indices.delete(
    index = index_name,
)

es.indices.create(
    index = index_name,
    body = index_definition
)

{u'acknowledged': True}

Create some data

In [3]:

import random

SAMPLE_ITEMS=100
SAMPLE_FIELDS=100
index_config = { 
    "_type"  : "apartment", 
    "_index" : 'apartments', 
}
records = []
cities  = ['city_{}'.format(x) for x in range(SAMPLE_FIELDS+1)]
streets = ['street_{}'.format(x) for x in range(SAMPLE_FIELDS+1)]
zips    = ['zip_{}'.format(x) for x in range(SAMPLE_FIELDS+1)]
emails  = ['email_{}'.format(x) for x in range(SAMPLE_FIELDS+1)]
coordinates = [
    {'lat' : x, 'lon':x}
    for x in range(-1-SAMPLE_FIELDS/2,1+SAMPLE_FIELDS/2)
]
for x in range(SAMPLE_ITEMS):
    r = {
        "city" : cities[random.randint(0,SAMPLE_FIELDS)],
        "coordinates" :  coordinates[random.randint(0,SAMPLE_FIELDS)],
        "email" : emails[random.randint(0,SAMPLE_FIELDS)],
        "reservations" : [],
        "street" : streets[random.randint(0,SAMPLE_FIELDS)],
        "zip" : zips[random.randint(0,SAMPLE_FIELDS)],
        "id" : x
    }
    records.append(r)


Lets generate some reservations

In [4]:

#10 will have one booking
for x in records[-10:]:
    x['reservations'] = [
        {
            'start_date' : '2017-01-01',
            'end_date'   : '2017-01-10',
        }
    ]


#5 will have 2 consecutive booking
for x in records[-5:]:
    x['reservations'].append(
        {
            'start_date' : '2017-01-10',
            'end_date'   : '2017-01-15',
        }
    )

#one will have a free time between 2017-01-02 - 2017-01-10
records[-1]['reservations'] = [
    {
        'start_date' : '2017-01-01',
        'end_date'   : '2017-01-02',
    },
    {
        'start_date' : '2017-01-12',
        'end_date'   : '2017-01-15',
    },
]


Insert them into ES fresh index

In [5]:

kwargs = {
    'body' : []
}

for idx,r in enumerate(records):
    _index_config = dict(index_config)
    _index_config['_id'] = idx
    kwargs['body'].append({'index' : _index_config})
    kwargs['body'].append(r)

es.bulk(**kwargs)

{u'errors': False,
 u'items': [{u'index': {u'_id': u'0',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'1',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'2',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'3',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'4',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'5',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'6',
    u'_index': u'apartments',
    u'_type': u'apartment',
    u'_version': 1,
    u'status': 201}},
  {u'index': {u'_id': u'7',
    u'_index': u'apartments',
    u'_type': u'a

And finally we perform the desired query.
There's some key points. 
We have to list free apartments and those apartment that will be available in the desired period (start_date, end_date variables)
So it should be a or query: free_aparments or available_aparments

The free apartments (those that haven't any value in `reservations` field) should be easy to query with a missing filter, but this is a nested field and we have to deal with. If we perform the query with a `missing` filter all docs will be returned. It's weird but it happens.
Here there's the explained solution: 
https://gist.github.com/Erni/7484095
and here is the issue:
https://github.com/elastic/elasticsearch/issues/3495
The gist snnipet works with all elasticsearch versions.

The other part of the `or` query are available apartments. I've solved this part performing a not query. Return me those apartments that NOT have a reservation, thought a list of `range` that match with those aparments that do have a reservation and then negate the result using `must_not` filter

In [6]:

start_date = "2017-01-04"
end_date   = "2017-01-09"

elasticsearch_query = {
    "query": {
        "filtered": {
            "filter": {
                "bool": {
                    "should": [
                        {
                            "nested": {
                                "filter": {
                                    "bool": {
                                        "must_not" : [
                                            {
                                                "range": {
                                                    "start_date": {
                                                        "gte" : start_date, 
                                                        "lt" :end_date
                                                    }
                                                }
                                            },
                                            {
                                                "range": {
                                                    "end_date": {
                                                        "gte" : end_date, 
                                                        #"lte" :end_date
                                                    }
                                                }
                                            }
                                        ]
                                    }
                                }, 
                                "path": "reservations"
                            }
                        },
                        {
                            #{ "missing" : { "field" : "reservations"} }
                            "not": {
                                "nested": {
                                    "path": "reservations",
                                    "filter": {
                                        "match_all": {}
                                    }
                                }
                            }
                        }
                    ],
                }
            }
        },
    }, 
    "sort" : {"id":"desc"}
}

es_kwargs = { 
    "doc_type"  : "apartment", 
    "index" : 'apartments', 
    "body" : elasticsearch_query
}

res = es.search(**es_kwargs)
for r in res['hits']['hits']:
    print r['_source'].get('reservations',[])

[{u'start_date': u'2017-01-01', u'end_date': u'2017-01-02'}, {u'start_date': u'2017-01-12', u'end_date': u'2017-01-15'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]


So here is the result. An available apartment plus free apartments.